In [1]:
import matplotlib.pylab as plt
import numpy as np
import matplotlib.cm as cm
import cmocean.cm as cmo
import intake

import numcodecs
try:
    import gribscan
except:
    %pip install gribscan
    import gribscan
import cfgrib
import gc

from getpass import getuser # Libaray to copy things
from pathlib import Path # Object oriented libary to deal with paths
from dask.utils import format_bytes
from dask.diagnostics import ProgressBar

from distributed import Client, progress, wait # Libaray to orchestrate distributed resources
#from dask_jobqueue import SLURMCluster # Setting up distributed memories via slurm
import numpy as np # Pythons standard array library
import xarray as xr # Libary to work with labeled n-dimensional data
import dask # Distributed data libary
import dask.distributed
import multiprocessing
from subprocess import run, PIPE
import sys
import os
import warnings
warnings.filterwarnings(action='ignore')
dask.config.config.get('distributed').get('dashboard').update({'link':'{JUPYTERHUB_SERVICE_PREFIX}/proxy/{port}/status'})
import pandas as pd
import eccodes


In [2]:
cat = intake.open_catalog("https://data.nextgems-h2020.eu/catalog.yaml")




In [3]:
#print(data.variables)

In [4]:

#print(list(cat.IFS['IFS_9-FESOM_5-production']))


In [5]:
#####calculate the temporal mean
def calculate_temporal(data,year,month,day,function,latitude,longitude):
    indexer = ((data.lat>=latitude[0]) & (data.lat<=latitude[1]) & (data.lon>=longitude[0]) & (data.lon<=longitude[1])).compute()
    data = data.where(indexer,drop=True)
    if function == 'precip':
        data = data.resample(time='1D').sum() * 1000
        data = data.where(data.time.dt.year.isin(year), drop=True)
        data = data.where(data.time.dt.month.isin(month), drop=True)
        data = data.where(data.time.dt.day.isin(day), drop=True)
        data.attrs['units'] = 'mm/day' 
    if function == 'tmean':
        data = data.where(data.time.dt.year.isin(year), drop=True)
        data = data.where(data.time.dt.month.isin(month), drop=True)
        data = data.where(dsel.time.dt.day.isin(day), drop=True)
        data = data.resample(time='1D').mean()
        data = data - 273.15
        data.attrs['units'] = 'C'     
    if function == 'tmax':
        data = data.where(data.time.dt.year.isin(year), drop=True)
        data = data.where(data.time.dt.month.isin(month), drop=True)
        data = data.where(dsel.time.dt.day.isin(day), drop=True)
        data = data.resample(time='1D').max()
        dsel = data - 273.15
        data.attrs['units'] = 'C'     
    #days = np.arange(1,32,1)
    data = data.load()

    return data

In [6]:
#####function that uses all the functions.
def temporal_mean(dataset,year,month,day,function,latitude,longitude):
# If you want to change the longitude range from 0:360 to -180:180 to make clipping easier around the prime  meridian
    dataset['lon_adjusted'] = xr.where(dataset.lon > 180, dataset.lon - 360, dataset.lon)
    dataset['lon'] = dataset['lon_adjusted']
    dataset = dataset.reset_coords(names='lon_adjusted', drop=True)
    
    ###calculate
    calc_var = calculate_temporal(dataset,year,month,day,function,latitude,longitude)
    return calc_var
    #del var_final
    del dataset
    del cal_var
    gc.collect()

In [7]:

res = 'IFS_9-FESOM_5'
var = ['tp'] #for cape, change to 2D_1h_0.25deg. precip = tp temperature = 2t
function = 'precip'
#months = [1,2,3,4,5,6,7,8,9,10,11,12]
months = [2,3]
#years = np.arange(2020, 2049)
years = [2021]
#days = np.arange(1,32,1)
days = np.arange(28,29,1)

latitude = [35,45] #Choose your own lat/lon range 
longitude = [-11,5]

#IFS_9-FESOM_5-production
#IFS_9-FESOM_5-production-hist
data = cat.IFS['IFS_9-FESOM_5-production']['2D_hourly_healpix512'](chunks={}).to_dask().drop_duplicates('time', keep='last') # load H512, hourly; and exclude time duplicates containing NaNs

dataset = data[var]

In [ ]:

variable = temporal_mean(dataset,years,months,days,function,latitude,longitude)


In [ ]:
print(variable.time)